In [3]:
import numpy as np
import math
from tqdm import tqdm
import pandas as pd
import pickle

from sklearn.mixture import GaussianMixture

from scipy.stats import t

from graspy.embed import JointRDPG
from graspy.cluster import GaussianCluster, KMeansCluster
from graspy.utils import symmetrize

from mgcpy.independence_tests.mgc import MGC
from mgcpy.independence_tests.dcorr import DCorr
from mgcpy.independence_tests.rv_corr import RVCorr

from simulations import rho_ER_marg, rho_sbm_marg, rho_sbm_diff_block
from utils import estimate_block_assignment, block_permute, sort_graph, identity, sbm_params, to_distance_mtx, triu_no_diag

In [2]:
def dcorr_ttest_power(sim_func, mc=500, alpha=0.05, given_blocks=False, blocks=None, **kwargs):
    # power for any test that builds on distance matrices
    # can use dcorr / mgc
    pval_array = np.zeros(mc)
    for i in range(mc):
        A, B = sim_func(**kwargs) 
        dcorr = DCorr()
        pval, _ = dcorr.p_value(
            matrix_X=triu_no_diag(A), matrix_Y=triu_no_diag(B))
        pval_array[i] = pval
    power = np.where(pval_array < alpha)[0].shape[0] / mc
    return power

In [4]:
def ttest_power(sim_func, mc=500, alpha=0.05, **kwargs):
    # power for any test that builds on distance matrices
    # can use dcorr / mgc
    pval_array = np.zeros(mc)
    for i in range(mc):
        A, B = sim_func(**kwargs) 
        test = RVCorr(which_test='pearson')
        pval, _ = test.p_value(
            matrix_X=triu_no_diag(A), matrix_Y=triu_no_diag(B))
        pval_array[i] = pval
    power = np.where(pval_array < alpha)[0].shape[0] / mc
    return power

In [10]:
A, B = rho_ER_marg(rho=0.1, p=0.5, q=0.5, n=100)

In [14]:
triu_no_diag(B).shape

(4950, 1)

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [3]:
def process_parallel_output(graph_type):
    n_arr = np.linspace(10, 100, 10, dtype=int)
    rho_arr = np.array([0, 0.1, -0.1])
    power = np.zeros((rho_arr.shape[0], n_arr.shape[0]))
    with open('../../results/rho_{}_power_parallel_ttest.pkl'.format(graph_type), 'rb') as f:
        results = pickle.load(f)

    count = 0
    for j, rho in enumerate(rho_arr):
        for k, n in enumerate(n_arr):
            power[j, k] = results[count][2]
            count += 1
    with open('../../results/rho_{}_power_ttest.pkl'.format(graph_type), 'wb') as f:
        pickle.dump(power, f)

In [5]:
process_parallel_output('er')

In [8]:
graph_type = 'sbm'
with open('../../results/rho_{}_power_parallel_ttest.pkl'.format(graph_type), 'rb') as f:
    results = pickle.load(f)

In [9]:
results

[(0.0, 10, 0.002),
 (0.0, 20, 0.592),
 (0.0, 30, 0.894),
 (0.0, 40, 1.0),
 (0.0, 50, 1.0),
 (0.0, 60, 1.0),
 (0.0, 70, 1.0),
 (0.0, 80, 1.0),
 (0.0, 90, 1.0),
 (0.0, 100, 1.0),
 (0.1, 10, 0.93),
 (0.1, 20, 0.856),
 (0.1, 30, 1.0),
 (0.1, 40, 1.0),
 (0.1, 50, 1.0),
 (0.1, 60, 1.0),
 (0.1, 70, 1.0),
 (0.1, 80, 1.0),
 (0.1, 90, 1.0),
 (0.1, 100, 1.0),
 (-0.1, 10, 0.61),
 (-0.1, 20, 0.142),
 (-0.1, 30, 0.29),
 (-0.1, 40, 0.696),
 (-0.1, 50, 0.82),
 (-0.1, 60, 0.92),
 (-0.1, 70, 0.972),
 (-0.1, 80, 0.982),
 (-0.1, 90, 1.0),
 (-0.1, 100, 1.0)]